In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import folium

import time
import datetime as dt
from datetime import timedelta, datetime, timezone
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

In [2]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
rse_coord = '도솔대교_RSE_위도경도.csv'

df = pd.read_csv(rse_coord)

map = folium.Map(tiles = "OpenStreetMap")

for index, row in df.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = 3, weight = 3, tooltip = row['rse_id']+row['node_nm'], color='red').add_to(map)
    
map.fit_bounds(map.get_bounds())

#map

In [4]:
name_id = {}
id_name = {}
id_lat = {}
id_long = {}

for index, row in df.iterrows():
    name_id[row['node_nm']] = row['rse_id']
    id_name[row['rse_id']] = row['node_nm']
    id_lat[row['rse_id']] = row['latitude']
    id_long[row['rse_id']] = row['longitude']
#print(rse_id)

In [5]:
class Node:
    def __init__(self, orig, orig_id, dest, dest_id, direction):
        self.origin = orig
        self.orig_id = orig_id
        self.dest = dest
        self.dest_id = dest_id
        self.direction = direction
        
#도솔대교 Node
Links = ['옥녀봉삼거리,도안동로입구','도안동로입구,옥녀봉삼거리',
         '옥녀봉네거리,옥녀봉삼거리','옥녀봉삼거리,옥녀봉네거리',
         '옥녀봉네거리,도솔네거리','도솔네거리,옥녀봉네거리',
         '도솔네거리,안골네거리','안골네거리,도솔네거리',
         '옥녀봉네거리,위더스타워네거리','위더스타워네거리,옥녀봉네거리',
         '도로교통공단대전충남지부앞,위더스타워네거리','위더스타워네거리,도로교통공단대전충남지부앞',
         '옥녀봉네거리,원신흥로입구','원신흥로입구,옥녀봉네거리',
         '원신흥로입구,원신흥북로종점','원신흥북로종점,원신흥로입구',
         '도안호반베르디움201동삼거리,원신흥북로종점','원신흥북로종점,도안호반베르디움201동삼거리',
         '도안호반베르디움201동삼거리,진터지하차도','진터지하차도,도안호반베르디움201동삼거리',
         '도안호반베르디움201동삼거리,용반네거리','용반네거리,도안호반베르디움201동삼거리',
         '유성네거리,계룡로132번길','계룡로132번길,유성네거리',
         '계룡로132번길,용반네거리','용반네거리,계룡로132번길',
         '용반네거리,만년교네거리','만년교네거리,용반네거리',
         '만년교네거리,월평초교삼거리','월평초교삼거리,만년교네거리',
         '갑천대교네거리,월평초교삼거리','월평초교삼거리,갑천대교네거리',
         '갑천대교네거리,갑천네거리','갑천네거리,갑천대교네거리',
         '만년교네거리,도안호반베르디움201동삼거리','도안호반베르디움201동삼거리,만년교네거리',
         '만년교네거리,원신흥로입구','원신흥로입구,만년교네거리']

Names = []

for i in Links:
    j = i.split(",")[0]
    k = i.split(",")[1]
    if(j not in Names):
        Names.append(j)
    if(k not in Names):
        Names.append(k)

Names.sort()
        
Nodelist = []

for i in Links:
    j = i.split(",")[0]
    k = i.split(",")[1]
    Nodelist.append(Node(j, name_id[j], k, name_id[k], j+'->'+k))

In [6]:
filenames23 = []
filenames24 = []

year = ['2023', '2024']
month = ['2월', '3월', '4월']
hours = ['0708', '0809', '0709', '1213', '1314', '1214', '1718', '1819', '1719']
peak = ['오전첨두', '비첨두', '오후첨두']

for y in year:
    for m in month:
        for i in range(3):
            for k in range(3):
                filename = y + '_' + m + '_' + peak[i] + '_' + hours[3*i + k] + '.csv'
                if(y == '2023'):
                    filenames23.append(filename)
                if(y == '2024'):
                    filenames24.append(filename)
                    

In [10]:
dates = ['2월', '3월','4월']
times = ['오전첨두', '비첨두', '오후첨두']
hours = [['07:00~08:00', '08:00~09:00', '07:00~09:00'],
         ['12:00~13:00', '13:00~14:00', '12:00~14:00'],
         ['17:00~18:00', '18:00~19:00', '17:00~19:00']]
dates_times_hours = []

for d in dates:
    for i in range(3):
        time = times[i]
        hour = hours[i]
        for t in hour:
            dates_times_hours.append(d+'_'+time+'_'+t)

print(dates_times_hours)

### filenames

#filenames23 = ['2023_4월_오전첨두_0708.csv', '2023_4월_오전첨두_0809.csv', '2023_4월_오전첨두_0709.csv', 
#               '2023_4월_비첨두_1213.csv', '2023_4월_비첨두_1314.csv', '2023_4월_비첨두_1214.csv',
#               '2023_4월_오후첨두_1718.csv', '2023_4월_오후첨두_1819.csv', '2023_4월_오후첨두_1719.csv']
#filenames24 = ['2024_4월_오전첨두_0708.csv', '2024_4월_오전첨두_0809.csv', '2024_4월_오전첨두_0709.csv', 
#               '2024_4월_비첨두_1213.csv', '2024_4월_비첨두_1314.csv', '2024_4월_비첨두_1214.csv',
#               '2024_4월_오후첨두_1718.csv', '2024_4월_오후첨두_1819.csv', '2024_4월_오후첨두_1719.csv']


###

dfdict = {}

t = 0
for dth in dates_times_hours:
    result = pd.DataFrame(columns = ['route_id', 'origin_RSE_id', 'destin_RSE_id', 'lon_origin', 'lat_origin', 'lon_destin', 
                                     'lat_destin', 'movers_2023', 'movers_2024', 'avg_speed_2023', 'avg_speed_2024',
                                      'avg_time_2023', 'avg_time_2024', 
                                      'movers_diff', 'avg_speed_diff', 'avg_time_diff', 'direction'])
    df23 = pd.read_csv('도솔대교_정책보고서_RSE데이터/' + filenames23[t])
    df24 = pd.read_csv('도솔대교_정책보고서_RSE데이터/' + filenames24[t])
    
    i = 0
    for node in Nodelist:
        row23 = df23.loc[(df23['start_rse'] == node.orig_id) & (df23['end_rse'] == node.dest_id)]
        row24 = df24.loc[(df24['start_rse'] == node.orig_id) & (df24['end_rse'] == node.dest_id)]
        
        route_id = i
        origin_RSE_id = node.orig_id
        destin_RSE_id = node.dest_id
        lon_origin = id_long[node.orig_id]
        lat_origin = id_lat[node.orig_id]
        lon_destin = id_long[node.dest_id]
        lat_destin = id_lat[node.dest_id]
        
        try:
            movers23 = row23['frequency'].iloc[0]
        except:
            movers23 = 0
        try:
            movers24 = row24['frequency'].iloc[0]
        except:
            movers24 = 0
        try:
            avg_speed23 = row23['avg_speed'].iloc[0]
        except:
            avg_speed23 = 0
        try:
            avg_speed24 = row24['avg_speed'].iloc[0]
        except:
            avg_speed24 = 0
        try:
            avg_time23 = row23['avg_time'].iloc[0]
        except:
            avg_time23 = 0
        try:
            avg_time24 = row24['avg_time'].iloc[0]
        except:
            avg_time24 = 0
        
        try:
            movers = row24['frequency'].iloc[0] - row23['frequency'].iloc[0]
        except:
            movers = 0
        
        try:
            avg_speed = row24['avg_speed'].iloc[0] - row23['avg_speed'].iloc[0]
        except:
            avg_speed = 0
        
        try:
            avg_time = row24['avg_time'].iloc[0] - row23['avg_time'].iloc[0]
        except:
            avg_time = 0
        
        direction = node.direction
        
        data = [{'route_id' : route_id, 'origin_RSE_id' : origin_RSE_id, 'destin_RSE_id' : destin_RSE_id,
                 'lon_origin' : lon_origin, 'lat_origin' : lat_origin, 'lon_destin' : lon_destin,
                 'lat_destin' : lat_destin, 'movers_2023' : movers23, 'movers_2024' : movers24,
                 'avg_speed_2023' : avg_speed23, 'avg_speed_2024' : avg_speed24,
                 'avg_time_2023' : avg_time23, 'avg_time_2024' : avg_time24,
                 'movers_diff' : movers, 'avg_speed_diff' : avg_speed, 'avg_time_diff' : avg_time,
                 'direction' : direction}]
        
        resapp = pd.DataFrame(data)
        
        result = pd.concat([result, resapp], ignore_index=True)
        
        i += 1
        
    dfdict[dth] = result
    
    result_name = 'results/' + dth + '.csv'
    result_name = result_name.replace(':', '')
    result_name = result_name.replace('00', '')
    result.to_csv(result_name, index=False, encoding='utf-8-sig')
    
    t += 1

['2월_오전첨두_07:00~08:00', '2월_오전첨두_08:00~09:00', '2월_오전첨두_07:00~09:00', '2월_비첨두_12:00~13:00', '2월_비첨두_13:00~14:00', '2월_비첨두_12:00~14:00', '2월_오후첨두_17:00~18:00', '2월_오후첨두_18:00~19:00', '2월_오후첨두_17:00~19:00', '3월_오전첨두_07:00~08:00', '3월_오전첨두_08:00~09:00', '3월_오전첨두_07:00~09:00', '3월_비첨두_12:00~13:00', '3월_비첨두_13:00~14:00', '3월_비첨두_12:00~14:00', '3월_오후첨두_17:00~18:00', '3월_오후첨두_18:00~19:00', '3월_오후첨두_17:00~19:00', '4월_오전첨두_07:00~08:00', '4월_오전첨두_08:00~09:00', '4월_오전첨두_07:00~09:00', '4월_비첨두_12:00~13:00', '4월_비첨두_13:00~14:00', '4월_비첨두_12:00~14:00', '4월_오후첨두_17:00~18:00', '4월_오후첨두_18:00~19:00', '4월_오후첨두_17:00~19:00']
